# **CxC III - Infinite Investments Challenge**
### By the Secret Agents - Thirandie, Deepika, Lisa, Andy

Hi Judges! This is our first times ever interacting with anything related to Data Science, so we are just happy to be here and to learn more about the subject!

We have seperated our notebook into a few sections:
1. Preliminary Thoughts and Research
2. Data Preprocessing
3. Running the Model

Our process was to first layout our data pipeline, running through all the steps earlier and getting to running the model and getting a Cross Validation score. Then, we went through all the steps again and it on the test dataset simotaneously.

## Part 1: Preliminary Thoughts and Research

In [ ]:
# From Starting Docs - Importing Relevant history.csv from downloaded location in Personal Drive

import pandas as pd;
df = pd.read_csv("./history.csv")
test = pd.read_csv("./test.csv")

Let's get some quick summary statistics on our history data set.

In [ ]:
# This tells us the rows and columns there are in the dataset
df.shape

In [ ]:
# As mentioned in the provided Python notebook, there is a lot more Churned customers than unchurned ones.
df.groupby(['label']).id.agg('count')

In [ ]:
# Another notable feature about this dataset is the amount of empty values there are - some columns have little to no data at all!
pd.options.display.max_rows = 128
df.isna().sum()

In [ ]:
# In Fact, there is no entry in this file without at least 1 empty column!
df_no_null = df.dropna()
df_no_null

Considering all the summary statistics, we have come to the following conclusions:

1. The missing values have to be dealt with, since without it there isn't much to do at all. We plan to deal with this by using the [MICE algorithm](https://medium.com/@brijesh_soni/topic-9-mice-or-multivariate-imputation-with-chain-equation-f8fd435ca91#:~:text=MICE%20stands%20for%20Multivariate%20Imputation,produce%20a%20final%20imputed%20dataset.) for numerical and date data, and to just use the mode for categorical data. (We recognize that there are a lot of missing categorical data, so the mode might not be fully accurate - however it is reasonable enough a strategy)
2. As for the imbalanced dataset, we plan to take the advice given in the provided notebook and attempt to balance the dataset with Churn/No Churn entries before training the model. Fortunately, because Python, there does seem to be packages that handle this for us - namely [Imbalanced Learn](https://imbalanced-learn.org/stable/index.html)
3. To further reduce variance and to combat the effects of potentially errorenous preprocessing we will employ the Random Forest Classifier, as it randomly picks between columns and rows of data to use. This should hopefully improve the model's accuracy and recall.

## Part 2: Data Preprocessing

### Section 0: The Types of Columns we are dealing with
Before starting, we should replace unify all NaNs with `np.nan` for unifying reasons. We will also look at each of the columns to determine its type.

In [ ]:
import numpy as np
null_mask = df.isna()
df[null_mask] = np.NaN

test_null_mask = test.isna()
test[test_null_mask] = np.NaN

df.head(3)

It looks the same, but we are assured that all the NaNs are now actually NaNs. As for the columns,

In [ ]:
# We determine which of the columns are numerical vs categorical with the following code:

# Our First Filter splits all float/integer columns from the others. However, the categorical ones are still mixed with date data etc.
categorical_cols = [
    col for col in df.columns
    if df[col].dtype not in ["float64", "int64"] # removes: float, integer columns
]
numerical_cols = [
    col for col in df.columns
    if df[col].dtype in ["float64", "int64"]
]

print("Numerical Columns: ", numerical_cols)
print("Categorical Columns: ", categorical_cols)

print("Number of numerical columns in the DataFrame:", len(numerical_cols))
print("Number of categorical columns in the DataFrame:", len(categorical_cols))

Nominal columns are those that have words or a few categories in them. Notably, some of these columns contain names or something similar and so would have over thousands of categories. However, since there are a few entries with a lot of one type of entry it still would be important to keep that data in... to combat this we can introduce an additional category for 'infrequent' if there are not enough entries of that type. This value will be manually determined.

In [ ]:
categorical_cols_nominals = ['type_code', 'country_code', 'currency_code', 'debit_code', 'branch', 'retail_plan', 'language_code', 'sss_agent', 'terminal_code', 'iso_funds_code',
                           'dup_trip_quad_code',  'special_tag', 'non_plan_book_value_flag',
                           'risk_tolerance', 'investment_objective', 'last_maintenance_user',
                            'retail_last_maintenance_user', 'arp_pension_origin', 'conjunction', 'loan_limit_override', 'special_fee_code']
print("Number of categoricalcols_nominal in DataFrame:", len(categorical_cols_nominals))

Nominal columns are those that have words or a few categories in them. Notably, some of these columns contain names or something similar and so would have over thousands of categories. However, since there are a few entries with a lot of one type of entry it still would be important to keep that data in... to combat this we can introduce an additional category for 'infrequent' if there are not enough entries of that type. This value will be manually determined.

In [ ]:
categorical_cols_bools = ['is_registered', 'is_active', 'tms_settlement_location',
                         'net_of_fees', 'fee_paid_separately', 'custody_fee_withdrawal',
                         'is_fee_exempt', 'include_client_consolidation', 'use_client_address',
                         'is_spousal',  'is_arp_locked', 'sss_location', 'sss_type', 'use_hand_delivery',
                          'use_mail', 'share_name_address_to_issuer', 'shareholder_instructions_received',
                          'rrsp_limit_reached',  'is_portfolio_account', 'has_no_min_commission',
                          'is_tms_eligible', 'is_agent_bbs_participant', 'is_parameters_account',
                          'is_spousal_transfer', 'spousal_age_flag', 'has_multiple_name',
                          'discretionary_trading_authorized', 'shareholder_language', 'title',
                          'function_code',  'receive_general_mailings', 'has_discrete_auth',
                          'is_non_objecting_beneficial_owner', 'is_objecting_to_disclose_info',
                          'consent_to_pay_for_mail', 'consent_to_email_delivery',
                          'has_received_instruction', 'is_broker_account',
                          'is_inventory_account', 'is_gl_account', 'is_control_account',
                          'is_extract_eligible', 'is_pledged', 'is_resp',
                          'use_original_date_for_payment_calc',  'is_family_resp',
                          'is_hrdc_resp',  'is_plan_grandfathered', 'is_olob', 'visible_in_reports', 'is_midwest_clearing_account']
print("Number of categoricalcols_bools in DataFrame:", len(categorical_cols_bools))

Boolean columns are those that are simple true and falses, which is nice! Of course the representation of T/F apparently differs but that is fine

In [ ]:
categorical_cols_date = ['last_trade_date', 'inception_date', 'last_update_date', 'last_maintenance_time',  'non_calendar_year_end',
                         'plan_effective_date', 'plan_end_date',  'rrif_original_date',
                        'inserted_at',  'updated_at', 'retail_last_maintenance_time' ]

print("Number of categoricalcols_date in DataFrame:", len(categorical_cols_date))

These Categorical Columns are actually Date columns, so we need to process them at a later date. From the names, it seems that turning them into numbers based off today's time would work as a transformation.

In [ ]:
label_col = ['label']

Last but not least, we have our label column, which is the output.

### Section 1: Converting the Date Columns
Let's hope Python's Date Libraries are strong enough so that we don't have to put too much effort into converting them

In [ ]:
for col in categorical_cols_date:
  # Convert date string into date object, then subtract today from it and get the seconds in between. If it can't be parsed right just ignore it
  # The hardest part about this was figuring out the timezone specifics but otherwise not bad
  df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)
  df[col] = (df[col] - pd.Timestamp.now().tz_localize('UTC')).dt.total_seconds()

  test[col] = pd.to_datetime(test[col], errors='coerce', utc=True)
  test[col] = (test[col] - pd.Timestamp.now().tz_localize('UTC')).dt.total_seconds()

df

### Section 2: Imputing the Non-numerical Columns
We are now ready to impute all missing values. To do this, we use the mode for nominal/boolean ones.

In [ ]:
# Import the Simple Imputer for those Categorical items
from sklearn.impute import SimpleImputer

nominalToImpute = df[categorical_cols_nominals]
test_nominalToImpute = test[categorical_cols_nominals]

Imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
Imputer.set_output(transform="pandas")

nominalToImpute = Imputer.fit_transform(nominalToImpute)
test_nominalToImpute = Imputer.transform(test_nominalToImpute)
nominalToImpute

In [ ]:
from sklearn.impute import SimpleImputer

boolsToImpute = df[categorical_cols_bools]
test_boolsToImpute = test[categorical_cols_bools]

boolImputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
boolImputer.set_output(transform="pandas")

boolsToImpute = boolImputer.fit_transform(boolsToImpute)
test_boolsToImpute = boolImputer.transform(test_boolsToImpute)
boolsToImpute

In [ ]:
# Merge them back in
df[categorical_cols_nominals] = nominalToImpute
df[categorical_cols_bools] = boolsToImpute

test[categorical_cols_nominals] = test_nominalToImpute
test[categorical_cols_bools] = test_boolsToImpute

df

### Section 3: One-Hot Encoding
Before imputing numerical values, we first one-hot encode everything else. However, as mentioned earlier, as some of the fields contain literally thousands of values we implement a method to get rid of categories that are not substantial.

Of course, since we backfilled our categorical data with the mode already, we need to be careful with what the cutoff should be. Manual checking from earlier suggests that maybe, uh, **500**? Also, we have to cap the max categories to 20 - otherwise we get like 1000 or so columns, which isn't great. Just so you know, 20 was deliberately chosen because of `type_code`

In [ ]:
from sklearn.preprocessing import OneHotEncoder

nominalOneHot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='infrequent_if_exist', min_frequency=500, max_categories=20)
nominalOneHot.set_output(transform="pandas")

nomOneHot = df[categorical_cols_nominals].astype(str)
nomOneHot = nominalOneHot.fit_transform(nomOneHot)

test_nomOneHot = df[categorical_cols_nominals].astype(str)
test_nomOneHot = nominalOneHot.transform(test_nomOneHot)

nomOneHot

In [ ]:
from sklearn.preprocessing import OneHotEncoder

booleanOneHot = OneHotEncoder(sparse_output=False, drop='if_binary', handle_unknown='infrequent_if_exist', min_frequency=500)
booleanOneHot.set_output(transform="pandas")

boolOneHot = df[categorical_cols_bools].astype(str)
boolOneHot = booleanOneHot.fit_transform(boolOneHot)

test_boolOneHot = df[categorical_cols_bools].astype(str)
test_boolOneHot = booleanOneHot.transform(test_boolOneHot)

boolOneHot

In [ ]:
print(nomOneHot.columns)
print(boolOneHot.columns)

In [ ]:
# Merge them back in
df = pd.concat([df, nomOneHot, boolOneHot], axis=1)
df = df.drop(categorical_cols_nominals, axis=1).drop(categorical_cols_bools, axis=1)

test = pd.concat([test, test_nomOneHot, test_boolOneHot], axis=1)
test = test.drop(categorical_cols_nominals, axis=1).drop(categorical_cols_bools, axis=1)

df

In [ ]:
test

### **Savepoint**
To Reduce on RAM (and because when we did this on Google Colab it ran out of RAM) ((Free Powerful Cloud Computing is a lie))(((literally it was faster to run these operations locally on a 7 year old laptop))) we will be making a quicksave of the data here - we still need to MICE it, which is a very labourious operation.

In [ ]:
df.to_csv("./history-onehot.csv", index=False)

In [ ]:
import pandas as pd;
df2 = pd.read_csv("./history-onehot.csv")
df2

### Section 4: MICE Imputing
Now for the rest of the rows. MICE time.

In [ ]:
# First, we need to cut off the label, since that isn't a number and MICE cannot handle it.
df_X = df2.drop(['label'], axis=1)
df_Y = df2['label']

In [ ]:
# Apparently its still a experimental feature within sklearn - but that won't stop us!
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

total_num_cols = np.concatenate((numerical_cols, categorical_cols_date))

numImpute = IterativeImputer(random_state=23950, initial_strategy='median')
numImpute.set_output(transform="pandas")

toNumImpute = df_X[total_num_cols]
toNumImpute = numImpute.fit_transform(toNumImpute)
toNumImpute

In [ ]:
# Merge it in
df_X = df_X.drop(total_num_cols, axis=1)

In [ ]:
df_X = pd.concat([df_X, toNumImpute], axis=1)
df_X

That should be all of the nulls taken care of! Hopefully this didn't corrupt our data too much...

In [ ]:
df3 = pd.concat([df_X, df_Y], axis=1)
df3

### Section 5: Encode the Y-Label as Well
Almost forgot - gotta label encode the Churn/No Churn!

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df3['label'] = label_encoder.fit_transform(df3['label'])

print(label_encoder.classes_)

# I know we basically already did this but just to be sure you know
X = df3.drop(["label"], axis=1)
y = df3["label"]

### **Savepoint**
It's important to save!

In [ ]:
X.to_csv('./history_X.csv')
y.to_csv('./history_y.csv')

## Part 3: Split training / testing data
Hopefully this is good! Going to follow the notebook.

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

# Oops, the X and y has an extra column I forgot to get rid of my bad
X = pd.read_csv('./history_X.csv')
X = X.drop(X.columns[0], axis=1)
y = pd.read_csv('./history_y.csv')
y = y.drop(y.columns[0], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Part 3: The Model
Its just the Bagged Random Forest Classifier that was taken off `imblearn` - just gonna quickly, uh, steal the provided code...

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import cross_val_score

classifer = BalancedRandomForestClassifier(max_features='log2', bootstrap=True, verbose=1, random_state=13, replacement=True)
cv_scores = cross_val_score(estimator=classifer, X=X_train, y=y_train, cv=5)
print(f"Average CV Score: {sum(cv_scores)/len(cv_scores)}")

In [ ]:
from sklearn.metrics import f1_score

classifer.fit(X_train, y_train)

y_pred = classifer.predict(X_val)
f1 = f1_score(y_val, y_pred)

print(f"F1 Score: {f1}")

Woah thats pretty good! Now to copy this onto the `test.csv`. We employ the same preprocessing steps, and then get the `classifer` (which we've just noticed is misspelt) to predict on the new dataset.

# Step 4: Doing it all again but on the `test.csv`

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_null_mask = test.isna()
test[test_null_mask] = np.NaN

In [ ]:
for col in categorical_cols_date:
  # Convert date string into date object, then subtract today from it and get the seconds in between. If it can't be parsed right just ignore it
  # The hardest part about this was figuring out the timezone specifics but otherwise not bad
  test[col] = pd.to_datetime(test[col], errors='coerce', utc=True)
  test[col] = (test[col] - pd.Timestamp.now().tz_localize('UTC')).dt.total_seconds()